## Read in preprocessed data

In [1]:
import pandas as pd

X_train = pd.read_pickle('../data/X_train_v3.pkl')
y_train = pd.read_pickle('../data/y_train_v3.pkl')

_______________________________________________________________________________________________________________________________

## Build Models

In [23]:
#Import cross validation and optimization
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import cross_validate
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV

#Import metrics
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score 
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve

#Import models
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.linear_model import SGDClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier
from multiprocessing import cpu_count

In [24]:
#Import warnings
import warnings

#Ignore warnings
warnings.filterwarnings('ignore')

In [25]:
#Experiment tracking
import mlflow
import mlflow.sklearn
import mlflow.xgboost

In [26]:


#Begin logging
mlflow.sklearn.autolog()

### Random Forest Classifier

In [27]:
#Set Experiment name
mlflow.set_experiment('Random Forest Classifier')

<Experiment: artifact_location='file:///home/ty/code/data_science/analysis/Titanic-Analysis/mlruns/431820994769440421', creation_time=1688500931603, experiment_id='431820994769440421', last_update_time=1688500931603, lifecycle_stage='active', name='Random Forest Classifier', tags={}>

In [33]:
mlflow.set_tag('Random Forest Classifier', 'Random Forest Classifier')
rf_clf = RandomForestClassifier()
rf_clf.fit(X_train, y_train)

y_pred = rf_clf.predict(X_train)


#Set run name
mlflow.set_tag('run_1', 'RandomForestClassifier')

2023/07/04 16:07:31 WARNING mlflow.sklearn: Failed to log training dataset information to MLflow Tracking. Reason: 'Series' object has no attribute 'flatten'
2023/07/04 16:07:34 WARNING mlflow.utils.autologging_utils: Encountered unexpected error during sklearn autologging: The following failures occurred while performing one or more logging operations: [MlflowException('Failed to perform one or more operations on the run with ID 4f62c32ba9ea415095a06b7723ae4f6a. Failed operations: [MlflowException("Changing param values is not allowed. Param with key=\'verbose\' was already logged with value=\'False\' for run ID=\'4f62c32ba9ea415095a06b7723ae4f6a\'. Attempted logging new value \'0\'.")]')]


### Gradient Boosting Classifier

In [29]:
#Create Gradient Boosting Classifier
# gb_clf = GradientBoostingClassifier()

# #Fit model
# gb_clf.fit(X_train, y_train)

# #Predict on training set
# y_pred = gb_clf.predict(X_train)

### SGD Classifier

In [30]:
# #Make SGD Classifier
# sgd_clf = SGDClassifier()

# #Fit model
# sgd_clf.fit(X_train, y_train)

# #Predict on training set
# y_pred = sgd_clf.predict(X_train)
